In [1]:
#Импорт библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from PIL import Image, ImageTk
from random import randrange

In [2]:
df=pd.read_csv('income_evaluation.csv')
df.shape

(32561, 15)

In [3]:
#убирает \n,\t
for x in df.columns:
    x_new=x.strip()
    df=df.rename(columns={x:x_new})

In [4]:
# Убираем лишние колонки
data = df.drop(['fnlwgt', 'capital-gain', 'capital-loss', 'education-num', ], axis=1)

In [5]:
for column in data[['workclass','education','marital-status','occupation','race', 'sex']]:
    data[column]=data[column].str.strip()

In [6]:
#Группирует страны по регионам
data=data.replace([' United-States',' Canada', ' Cuba',' Jamaica',' Dominican-Republic',' El-Salvador',' Guatemala',' Haiti',' Honduras',' Mexico',' Nicaragua',' Outlying-US(Guam-USVI-etc)',' Puerto-Rico',
                   ' Trinadad&Tobago'],'North America')
data=data.replace([' India',' Cambodia',' China',' Hong',' Iran',' Japan',' Laos',' Philippines',' Taiwan',' Thailand',' Vietnam'],'Asia')
data=data.replace([' Ecuador',' Columbia',' Peru'],'South America')
data=data.replace([' England',' France',' Germany',' Greece',' Holand-Netherlands',' Hungary',' Ireland',' Italy',' Poland',' Portugal',' Scotland',' Yugoslavia'],'Europe')
data=data.replace([' South',' ?'],'Other')

In [7]:
#Таблица на русском для графиков
data_plots=data.copy()
data_plots=data_plots.rename(columns={'age':'Возраст','workclass':'Работа','education':'Образование',
                                      'marital-status':'Семейное положение','occupation':'Профессия','relationship':'Отношения',
                                      'race':'Раса','sex':'Пол','hours-per-week':'Рабочие часы','native-country':'Регион',
                                      'income':'Доход'})

In [8]:
# Меняем качественные переменные на числа

wcl_titles=data['workclass'].unique()
educ_titles=data['education'].unique()
native_titles=data['native-country'].unique()
mst_titles=data['marital-status'].unique()
occ_titles=data['occupation'].unique()
rel_titles=data['relationship'].unique()
race_titles=data['race'].unique()
hpw_titles=data['hours-per-week'].unique()
sex_titles=data['sex'].unique()

In [9]:
# Работа
lb_workclass = preprocessing.LabelEncoder()
lb_workclass.fit(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov', 'Local-gov', '?','Self-emp-inc', 'Without-pay',
                  'Never-worked'])
data.iloc[:,1]=lb_workclass.transform(data.iloc[:,1])
wcl_list=['Правительство субъекта','Незарегистрированный предприниматель','Частная компания', 'Федеральное правительство',
          'Местное правительство','Другое','Зарегистрированный предприниматель','Без оплаты','Безработный']
wcl_dict={x:y for x,y in zip(wcl_titles,wcl_list)}

data_plots=data_plots.replace(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov', 'Local-gov', '?', 'Self-emp-inc',
                               'Without-pay', 'Never-worked'],wcl_list)


In [10]:
# Регион
lb_native = preprocessing.LabelEncoder()
lb_native.fit(['North America', 'Asia', 'Other', 'Europe', 'South America'])
data.iloc[:,9]=lb_native.transform(data.iloc[:,9])
native_list=['Северная Америка','Азия','Другое', 'Европа','Южная Америка']
native_dict={x:y for x,y in zip(native_titles,native_list)}

In [11]:
data_plots=data_plots.replace(['North America', 'Asia', 'Other', 'Europe', 'South America'],native_list)

In [12]:
# Образование
lb_educ = preprocessing.LabelEncoder()
lb_educ.fit(['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college', 'Assoc-acdm', 'Assoc-voc', '7th-8th',
             'Doctorate', 'Prof-school', '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])
data.iloc[:,2]=lb_educ.transform(data.iloc[:,2])
educ_list=['Бакалавр','Среднее образование','11 классов','Магистр','9 классов','Незаконченное высшее','Другое','СПО',
           '7-8 классов','Докторская','Профессиональная школа','5-6 классов','10 классов','1-4 класс','Дошкольное','12 классов']
educ_dict={x:y for x,y in zip(educ_titles,educ_list)}

In [13]:
data_plots=data_plots.replace(['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college', 'Assoc-acdm', 'Assoc-voc',
                               '7th-8th', 'Doctorate', 'Prof-school', '5th-6th', '10th','1st-4th', 'Preschool', '12th'],
                              educ_list)

In [14]:
# Семейное положение
lb_marital = preprocessing.LabelEncoder()
lb_marital.fit(['Never-married', 'Married-civ-spouse', 'Divorced', 'Married-spouse-absent', 'Separated', 'Married-AF-spouse',
                'Widowed'])
data.iloc[:,3]=lb_marital.transform(data.iloc[:,3])
mst_list=['Никогда не был(а) в браке','В браке с гражданским','Разведен(а)','В браке, раздельно от супруга',
          'В браке, раздельно от супруга (по правовой причине)','В браке с военноослужащим','Овдовевший(ая)']
mst_dict={x:y for x,y in zip(mst_titles,mst_list)}

In [15]:
data_plots=data_plots.replace(['Never-married', 'Married-civ-spouse', 'Divorced', 'Married-spouse-absent', 'Separated',
                               'Married-AF-spouse', 'Widowed'], mst_list)

In [16]:
# Профессия
lb_occup = preprocessing.LabelEncoder()
lb_occup.fit(['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners', 'Prof-specialty', 'Other-service', 'Sales',
              'Craft-repair', 'Transport-moving', 'Farming-fishing', 'Machine-op-inspct', 'Tech-support', '?',
              'Protective-serv', 'Armed-Forces', 'Priv-house-serv'])
data.iloc[:,4]=lb_occup.transform(data.iloc[:,4])
occ_list=['Администратор/секретарь','Руководитель','Уборщик','Профессиональная специальность','Услуги','Продажи',
          'Ремесло/ремонт','Перевозки','Сельское хозяйство/рыбная ловля','Оператор машины/станка','Техническая поддержка',
          'Другое','Служба защиты','Военные силы','Служба жилья']
occ_dict={x:y for x,y in zip(occ_titles,occ_list)}

In [17]:
data_plots=data_plots.replace(['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners', 'Prof-specialty', 'Other-service',
                               'Sales', 'Craft-repair', 'Transport-moving', 'Farming-fishing', 'Machine-op-inspct',
                                'Tech-support', '?', 'Protective-serv', 'Armed-Forces', 'Priv-house-serv'], occ_list)

In [18]:
# Отношение
lb_rel = preprocessing.LabelEncoder()
lb_rel.fit([' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried', ' Other-relative'])
data.iloc[:,5]=lb_rel.transform(data.iloc[:,5])
rel_list=['Без семьи','Муж','Жена','Есть ребенок','Не в браке','Другое']
rel_dict={x:y for x,y in zip(rel_titles,rel_list)}

In [19]:
data_plots=data_plots.replace([' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried', ' Other-relative'], rel_list)

In [20]:
# Раса
lb_race = preprocessing.LabelEncoder()
lb_race.fit(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other'])
data.iloc[:,6]=lb_race.transform(data.iloc[:,6])
race_list=['Белый(ая)','Черный(ая)','Азиат/тихоокеанского происхождения','Американские индейцы/эскимосы','Другое']
race_dict={x:y for x,y in zip(rel_titles,race_list)}

In [21]:
data_plots=data_plots.replace(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other'], race_list)

In [22]:
# Пол
lb_sex = preprocessing.LabelEncoder()
lb_sex.fit(['Male','Female'])
data.iloc[:,7]=lb_sex.transform(data.iloc[:,7])
sex_list=['Мужской','Женский']
sex_dict={x:y for x,y in zip(sex_titles,sex_list)}

In [23]:
data_plots=data_plots.replace(['Male','Female'], sex_list)

In [24]:
#Группирует числовые значения для графиков
data_plots_count=data_plots.copy()

In [25]:
data_plots_count = data_plots_count.replace(list(range(1,11)),'1-10')
data_plots_count = data_plots_count.replace(list(range(11,21)),'11-20')
data_plots_count = data_plots_count.replace(list(range(21,31)),'21-30')
data_plots_count = data_plots_count.replace(list(range(31,41)),'31-40')
data_plots_count = data_plots_count.replace(list(range(41,51)),'41-50')
data_plots_count = data_plots_count.replace(list(range(51,61)),'51-60')
data_plots_count = data_plots_count.replace(list(range(61,71)),'61-70')
data_plots_count = data_plots_count.replace(list(range(71,81)),'71-80')
data_plots_count = data_plots_count.replace(list(range(81,91)),'81-90')
data_plots_count = data_plots_count.replace(list(range(91,100)),'91-99')

In [26]:
X=data.iloc[:,:-1]
y=data[['income']]

In [27]:
# Тренировочные/тестовые данные
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=7)

In [28]:
# GUI
income_values = ['<=50K', '>50K']

##################
# Основное окно
##################
root = Tk()
root.geometry("700x800")
root.title('Анализ данных')
root.iconbitmap('small_icon.ico')
root.configure(background="white")

# Лого
logo = Image.open('img/icon.png')
new_logo = Image.new("RGBA", logo.size, "WHITE")
new_logo.paste(logo, (0, 0), logo)
new_logo = ImageTk.PhotoImage(new_logo)
logo_label = tk.Label(image=new_logo, bg="white")
logo_label.image = new_logo
logo_label.place(x=150, y=-40)

dice_pic = PhotoImage(file = 'img/dice.png')
result_pic = PhotoImage(file = 'img/but_result.png')
learn_pic = PhotoImage(file = 'img/but_learn.png')
data_pic = PhotoImage(file = 'img/but_data.png')
exit_pic = PhotoImage(file = 'img/but_exit.png')
write_pic = PhotoImage(file = 'img/but_write.png')
pie_pic = PhotoImage(file = 'img/but_pie.png')
hist_pic = PhotoImage(file = 'img/but_hist.png')
alg_pic = PhotoImage(file = 'img/but_alg.png')
knn_pic = PhotoImage(file = 'img/but_knn.png')
forest_pic = PhotoImage(file = 'img/but_forest.png')

# Подписи
Label(root, text="Возраст", font=('Candara', 13, 'bold'), bg="white").place(x=180, y=135)
Label(root, text="Категория работы", font=('Candara', 13, 'bold'), bg="white").place(x=100, y=180)
Label(root, text="Образование", font=('Candara', 13, 'bold'), bg="white").place(x=135, y=225)
Label(root, text="Семейное положение", font=('Candara', 13, 'bold'), bg="white").place(x=80, y=270)
Label(root, text="Профессия", font=('Candara', 13, 'bold'), bg="white").place(x=150, y=315)
Label(root, text="Отношение к семье", font=('Candara', 13, 'bold'), bg="white").place(x=90, y=360)
Label(root, text="Раса", font=('Candara', 13, 'bold'), bg="white").place(x=200, y=405)
Label(root, text="Пол", font=('Candara', 13, 'bold'), bg="white").place(x=200, y=450)
Label(root, text="Родной регион", font=('Candara', 13, 'bold'), bg="white").place(x=120, y=495)
Label(root, text="Рабочие часы в неделю", font=('Candara', 13, 'bold'), bg="white").place(x=50, y=540)
Label(root, text="Результат: ", font=('Candara', 14, 'bold'), bg="white", relief='solid', width=10).place(x=50, y=670)

# Переменные
age = IntVar()
wcl = StringVar()
wcl.set(wcl_list[0])
educ = StringVar()
educ.set(educ_list[0])
mst = StringVar()
mst.set(mst_list[0])
occ = StringVar()
occ.set(occ_list[0])
rel = StringVar()
rel.set(rel_list[0])
race = StringVar()
race.set(race_list[0])
sex = StringVar()
sex.set(sex_list[0])
native = StringVar()
native.set(native_list[0])
hpw = IntVar()
inc=StringVar()
inc.set(income_values[0])

c_age_var = IntVar()
c_wcl_var = IntVar()
c_educ_var = IntVar()
c_mst_var = IntVar()
c_occ_var = IntVar()
c_rel_var = IntVar()
c_race_var = IntVar()
c_sex_var = IntVar()
c_native_var = IntVar()
c_hpw_var = IntVar()

alg_var = StringVar()
alg_var.set('forest')

# Проверка энтри
def validate(P):
    if len(P) == 0:
        return True
    elif 0 <= len(P) <= 2 and P.isdigit():
        return True
    else:
        return False


vcmd = (root.register(validate), '%P')

# Виджеты
e_age = Entry(root, text=age, width=10, validate="key", validatecommand=vcmd)
e_age.place(x=250, y=135)
om_wcl = OptionMenu(root, wcl, *wcl_list)
om_wcl.config(bg="#cfebf0", font=('Candara', 12),activebackground="#cfebf0",borderwidth=0.5)
om_wcl["menu"].config(bg="#cfebf0")
om_wcl.place(x=250, y=180)
om_educ = OptionMenu(root, educ, *educ_list)
om_educ.config(bg="#cfebf0", font=('Candara', 12),activebackground="#cfebf0",borderwidth=0.5)
om_educ["menu"].config(bg="#cfebf0")
om_educ.place(x=250, y=225)
om_mst = OptionMenu(root, mst, *mst_list)
om_mst.config(bg="#cfebf0", font=('Candara', 12),activebackground="#cfebf0",borderwidth=0.5)
om_mst["menu"].config(bg="#cfebf0")
om_mst.place(x=250, y=270)
om_occ = OptionMenu(root, occ, *occ_list)
om_occ.config(bg="#cfebf0", font=('Candara', 12),activebackground="#cfebf0",borderwidth=0.5)
om_occ["menu"].config(bg="#cfebf0")
om_occ.place(x=250, y=315)
om_rel = OptionMenu(root, rel, *rel_list)
om_rel.config(bg="#cfebf0", font=('Candara', 12),activebackground="#cfebf0",borderwidth=0.5)
om_rel["menu"].config(bg="#cfebf0")
om_rel.place(x=250, y=360)
om_race = OptionMenu(root, race, *race_list)
om_race.config(bg="#cfebf0", font=('Candara', 12),activebackground="#cfebf0",borderwidth=0.5)
om_race["menu"].config(bg="#cfebf0")
om_race.place(x=250, y=405)
om_sex = OptionMenu(root, sex, *sex_list)
om_sex.config(bg="#cfebf0", font=('Candara', 12),activebackground="#cfebf0",borderwidth=0.5)
om_sex["menu"].config(bg="#cfebf0")
om_sex.place(x=250, y=450)
om_nat = OptionMenu(root, native, *native_list)
om_nat.config(bg="#cfebf0", font=('Candara', 12),activebackground="#cfebf0",borderwidth=0.5)
om_nat["menu"].config(bg="#cfebf0")
om_nat.place(x=250, y=495)
e_hpw = Entry(root, text=hpw, width=10, validate="key", validatecommand=vcmd)
e_hpw.place(x=250, y=540)


##################
# Функции
##################

# Проверка пустых значений
def check_number(var):
    if var == '':
        messagebox.showerror("Ошибка", "Необходимо заполнить все поля")
        return False
    else:
        return True


# Круговая диаграмма
def make_plot_pie(figure, frame, column_name, input_get, title):
    ax = figure.add_subplot(111)
    bar = FigureCanvasTkAgg(figure, frame)
    colors = ['#65c72e', '#3bb8d2']
    bar.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH)
    new_data = data_plots[data_plots[column_name] == input_get]
    new_data = pd.Series(new_data['Доход']).value_counts()
    ax.pie(new_data, labels=income_values, explode=[0, 0], autopct='%1.1f%%', radius=1, shadow=True, colors=colors)
    ax.set_title(title, fontsize=9)


# гистограмма по категориям
def make_countplot(figure,frame,column_name):
    countplot_list=[]
    unique_values=data_plots_count[column_name].unique()
    for val in unique_values:
        countplot_list.append(data_plots_count[data_plots_count[column_name]==val]['Доход'])
    ax = figure.add_subplot(111)
    bar = FigureCanvasTkAgg(figure, frame)
    bar.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH)
    p=figure.gca()
    p.hist(countplot_list,3,label=unique_values)
    p.legend()
    ax.set_title(column_name, fontsize=10)
    ax.set_xlabel('Доход')
    ax.set_ylabel('Количество людей')


# Проверка наличия данных с определенным параметром
def have_data(column_name, input_get):
    new_data = data_plots[data_plots[column_name] == input_get]
    if pd.Series(new_data['Доход']).value_counts().empty:
        return False
    else:
        return True


# Для чекбоксов
def toggle(var):
    var.set(not var.get())


# Прокрутка окна
def make_scrollbar(window, pad):
    main_frame = Frame(window)
    main_frame.pack(fill=BOTH, expand=1)

    my_canvas = Canvas(main_frame, height=600)
    my_canvas.pack(side=LEFT, fill=BOTH, expand=1, padx=pad)

    my_scrollbar = ttk.Scrollbar(main_frame, orient=VERTICAL, command=my_canvas.yview)
    my_scrollbar.pack(side=RIGHT, fill=Y)

    # Прокрутка скроллбара мышкой
    def _on_mouse_wheel(event):
        my_canvas.yview_scroll(-1 * int((event.delta / 120)), "units")

    my_canvas.configure(yscrollcommand=my_scrollbar.set)
    my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion=my_canvas.bbox("all")))
    my_canvas.bind_all("<MouseWheel>", _on_mouse_wheel)

    second_frame = Frame(my_canvas, width=700, height=600)
    second_frame.configure(background='white')

    my_canvas.create_window((0, 0), window=second_frame, anchor='nw')
    my_canvas.configure(background='white')
    main_frame.configure(background='white')

    return second_frame


# делает словарь и список для графиков
def collect_data_for_charts(age, wcl, educ, mst, occ, rel, race, sex, hpw, native, c_age_var, c_wcl_var, c_educ_var,
                            c_mst_var, c_occ_var,
                            c_rel_var, c_race_var, c_sex_var, c_hpw_var, c_native_var):
    chart_list_a = [['C возрастом ' + str(age.get()) + ' лет', age.get()], ['В категории:' + wcl.get(), wcl.get()],
                    ['С образованием: ' + educ.get(), educ.get()], ['С семейным положением:' + mst.get(), mst.get()],
                    ['В професии: ' + occ.get(), occ.get()], ['Относящихся к семье как: ' + rel.get(), rel.get()],
                    ['С расой:' + race.get(), race.get()], ['С полом: ' + sex.get(), sex.get()],
                    ['Работающих ' + str(hpw.get()) + ' часов в неделю:', hpw.get()],
                    ['Из региона: ' + native.get(), native.get()]]

    chart_dict = {x: y for x, y in zip(data_plots.columns.values, chart_list_a)}
    charts_to_make = []
    charts_list = [c_age_var.get(), c_wcl_var.get(), c_educ_var.get(), c_mst_var.get(), c_occ_var.get(),
                   c_rel_var.get(),
                   c_race_var.get(), c_sex_var.get(), c_native_var.get(), c_hpw_var.get()]

    for ind, elem in enumerate(charts_list):
        if elem == 1:
            charts_to_make.append(data_plots.columns.values[ind])

    return charts_to_make, chart_dict


# Индекс элемента
def get_index(arr, val):
    return arr.index(val.get())

def rand():
    e_age.delete(0,"end")
    e_age.insert(0, randrange(18, 91))
    
    wcl.set(wcl_list[randrange(0, 8)])
    educ.set(educ_list[randrange(0, 16)])
    mst.set(mst_list[randrange(0, 7)])
    occ.set(occ_list[randrange(0, 15)])
    rel.set(rel_list[randrange(0, 6)])
    race.set(race_list[randrange(0, 5)])
    sex.set(sex_list[randrange(0, 2)])
    native.set(native_list[randrange(0, 5)])
    
    e_hpw.delete(0,"end")
    e_hpw.insert(0, randrange(1, 100))
    


##################
# Окно с выбором графика
##################
def info():
    n = Toplevel()
    n.geometry("550x350")
    n.title('Анализ данных')
    n.iconbitmap('small_icon.ico')
    n.configure(background='white')

    Button(n, image = exit_pic, command=n.destroy, bg='#ffffff',borderwidth=0, activebackground="#ffffff").place(x=350, y=-5)

    c_age = Checkbutton(n, text="Возраст", font=('Candara', 12), bg='white', command=lambda: toggle(c_age_var)).place(
        x=245, y=50)
    c_wcl = Checkbutton(n, text="Категория работы", font=('Candara', 12), bg='white',
                        command=lambda: toggle(c_wcl_var)).place(x=245, y=75)
    c_educ = Checkbutton(n, text="Образование", font=('Candara', 12), bg='white',
                        command=lambda: toggle(c_educ_var)).place(x=245, y=100)
    c_mst = Checkbutton(n, text="Семейное положение", font=('Candara', 12), bg='white',
                        command=lambda: toggle(c_mst_var)).place(x=245, y=125)
    c_occ = Checkbutton(n, text="Профессия", font=('Candara', 12), bg='white', command=lambda: toggle(c_occ_var)).place(x=245, 
                                                                                                                        y=150)
    c_rel = Checkbutton(n, text="Отношение к семье", font=('Candara', 12), bg='white',
                        command=lambda: toggle(c_rel_var)).place(x=245, y=175)
    c_race = Checkbutton(n, text="Раса", font=('Candara', 12), bg='white', command=lambda: toggle(c_race_var)).place(x=245,
                                                                                                                     y=200)
    c_sex = Checkbutton(n, text="Пол", font=('Candara', 12), bg='white', command=lambda: toggle(c_sex_var)).place(x=245,y=225)
    c_native = Checkbutton(n, text="Родной регион", font=('Candara', 12), bg='white',
                           command=lambda: toggle(c_native_var)).place(x=245, y=250)
    c_hpw = Checkbutton(n, text="Рабочие часы в неделю", font=('Candara', 12), bg='white',
                        command=lambda: toggle(c_hpw_var)).place(x=245, y=275)

    Button(n, image = pie_pic, command=pie, bg='#ffffff',borderwidth=0,activebackground="#ffffff").place(x=25, y=40)

    Button(n, image = hist_pic, command=countplot, bg='#ffffff',borderwidth=0,activebackground="#ffffff").place(x=25, y=120)

    n.resizable(0, 0)
    n.mainloop()


##################
# Окно с круговой диаграммой
##################
def pie():
    try:
        pie_window = Tk()
        pie_window.geometry("600x700")
        pie_window.title('Анализ данных')
        pie_window.iconbitmap('small_icon.ico')
        pie_window.configure(background='white')

        second_frame = make_scrollbar(pie_window, 60)
        inc_share_label = tk.Label(second_frame, text='Распределение дохода у людей:', font=('Candara', 18, 'bold'),
                                   bg='white').pack()


        charts_to_make, chart_dict = collect_data_for_charts(age, wcl, educ, mst, occ, rel, race, sex, hpw, native,
                                                             c_age_var, c_wcl_var,
                                                             c_educ_var, c_mst_var, c_occ_var, c_rel_var, c_race_var,
                                                             c_sex_var,
                                                             c_native_var, c_hpw_var)

        for elem in charts_to_make:
            if have_data(elem, chart_dict[elem][1]):
                figure = plt.Figure(figsize=(4, 3), dpi=100)
                make_plot_pie(figure, second_frame, elem, chart_dict[elem][1], chart_dict[elem][0])

        pie_window.resizable(0, 0)
        pie_window.mainloop()
    except TclError:
        messagebox.showerror("Ошибка", "Необходимо заполнить все поля")
        pie_window.destroy()


##################
# Окно с гистограммой
##################
def countplot():
    try:
        count_window = Tk()
        count_window.geometry("700x650")
        count_window.title('Анализ данных')
        count_window.iconbitmap('small_icon.ico')
        count_window.configure(background='white')

        second_frame = make_scrollbar(count_window, 20)
        inc_share_label = tk.Label(second_frame, text='Распределение дохода у людей', font=('Candara', 18, 'bold'),
                                   bg='white').pack()

        charts_to_make, chart_dict = collect_data_for_charts(age, wcl, educ, mst, occ, rel, race, sex, hpw, native, c_age_var,
                                                 c_wcl_var, c_educ_var,
                                                 c_mst_var, c_occ_var, c_rel_var, c_race_var, c_sex_var, c_native_var,
                                                 c_hpw_var)
        for elem in charts_to_make:
            figure = plt.Figure(figsize=(7, 6))
            make_countplot(figure, second_frame, elem)


        count_window.resizable(0, 0)
        count_window.mainloop()
    except TclError:
        messagebox.showerror("Ошибка", "Необходимо заполнить все поля")
        count_window.destroy()

def add_row():
    row_window = Toplevel()
    row_window.geometry("450x250")
    row_window.title('Анализ данных')
    row_window.iconbitmap('small_icon.ico')
    row_window.configure(background='white')
    
    wcl_value = get_index(wcl_list, wcl)
    educ_value = get_index(educ_list, educ)
    mst_value = get_index(mst_list, mst)
    occ_value = get_index(occ_list, occ)
    rel_value = get_index(rel_list, rel)
    race_value = get_index(race_list, race)
    sex_value = get_index(sex_list, sex)
    native_value = get_index(native_list, native)
    x_test = [[age.get(), wcl_value, educ_value, mst_value, occ_value, rel_value, race_value, sex_value, hpw.get(),
               native_value]]
    
    r_var = StringVar()
    r_var.set(' <=50K')
    
    def add_to_df():
        x_test_ser=pd.Series(x_test)
        df_length = len(X_train)+1
        x_test_ser.fillna(0)
#         X_train.loc[df_length] = x_test_ser
        
        y_test_ser=pd.Series(r_var.get())
        df_length = len(y_train)+1
#         y_train.loc[df_length] = y_test_ser
        messagebox.showinfo("Добавление", "Запись успешно добавлена!")
        
    Label(row_window, text="Получаемый доход в год (USD)", font=('Candara', 14,'bold'), bg="white").place(x = 75, y=20)

    r1 = Radiobutton(row_window,text='Меньше 50 тысяч', variable=r_var, value=' <=50K', font=('Candara', 13),background='white')
    r2 = Radiobutton(row_window,text= 'Больше 50 тысяч', variable=r_var, value='>50K', font=('Candara', 13),background='white')
    r1.place(x=135, y=75)
    r2.place(x=135, y=110)
    
    Button(row_window, image = write_pic,command=add_to_df, bg='#ffffff', borderwidth=0,activebackground="#ffffff").place(x=20,
                                                                                                                          y=170)
    Button(row_window, image = exit_pic, command=row_window.destroy, bg='#ffffff',borderwidth=0, 
           activebackground="#ffffff").place(x=230,y=170)

    row_window.resizable(0, 0)
    row_window.mainloop()
    
def alg():
    alg_window = Toplevel()
    alg_window.geometry("300x300")
    alg_window.title('Анализ данных')
    alg_window.iconbitmap('small_icon.ico')
    alg_window.configure(background='white')
    
    def set_alg_var(x):
        alg_var.set(x)
        alg_window.destroy()
    
    Button(alg_window, image = knn_pic,command=lambda: set_alg_var('knn'), bg='#ffffff', borderwidth=0,
           activebackground="#ffffff").place(x=25,y=30)
    
    Button(alg_window, image = forest_pic,command=lambda: set_alg_var('forest'), bg='#ffffff', borderwidth=0,
           activebackground="#ffffff").place(x=25,y=140)

    alg_window.resizable(0, 0)
    alg_window.mainloop()
    

##################
# forest/knn
##################
def model():
    try:
        # Ввод данных
        wcl_value = get_index(wcl_list, wcl)
        educ_value = get_index(educ_list, educ)
        mst_value = get_index(mst_list, mst)
        occ_value = get_index(occ_list, occ)
        rel_value = get_index(rel_list, rel)
        race_value = get_index(race_list, race)
        sex_value = get_index(sex_list, sex)
        native_value = get_index(native_list, native)

        # Классифаер
        if alg_var.get()=='knn':
            classifier = KNeighborsClassifier(n_neighbors=9)
        else:
            classifier = RandomForestClassifier(n_estimators=19,max_features=4)
        classifier.fit(X_train, np.ravel(y_train))
        x_test = [[age.get(), wcl_value, educ_value, mst_value, occ_value, rel_value, race_value, sex_value, hpw.get(),
                   native_value]]
        y_pred = classifier.predict(x_test)
        print(type(x_test))

        # Вывод рез-та
        if y_pred[0] == ' <=50K':
            result_text = 'Меньше 50 тысяч в год'
        else:
            result_text = 'Больше 50 тысяч в год'

        proba = classifier.predict_proba(x_test).max() * 100
        Label(root, text=result_text, font=('Candara', 15, 'bold'), bg="white", width=20).place(x=159, y=670)
        Label(root, text='С вероятностью в ' + str(round(proba, 2)) + '%', font=('Candara', 13), bg="white",
              width=20).place(x=171, y=700)

    except TclError:
        messagebox.showerror("Ошибка", "Необходимо заполнить все поля")
        


# Кнопочки
Button(root, image = dice_pic, borderwidth=0,command=rand,bg='#ffffff',activebackground="#ffffff").place(x=40,y=115)

Button(root, image = alg_pic, borderwidth=0,command=alg,bg='#ffffff',activebackground="#ffffff").place(x=465,y=105)

Button(root, image = result_pic, command=model, bg='#ffffff',borderwidth=0,activebackground="#ffffff").place(x=50,y=580)

Button(root, image = learn_pic, command=info, bg='#ffffff',borderwidth=0,activebackground="#ffffff").place(x=250, y=580)

Button(root, image = data_pic, command=add_row, bg='#ffffff',borderwidth=0,activebackground="#ffffff").place(x=495, y=580)

Button(root, image = exit_pic, command=root.destroy, bg='#ffffff',borderwidth=0, activebackground="#ffffff").place(x=495, y=720)

root.resizable(0, 0)
root.mainloop()

<class 'list'>
